# Script writing for 5311 data

In [1]:
import numpy as np
import pandas as pd
from siuba import *
import altair as alt
import altair_saver
from calitp import *
from plotnine import *
import intake

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [2]:
import data_prep

In [3]:
#df = data_prep.load_grantprojects()
#vehicles = data_prep.load_vehiclesdata()
# agency_info = data_prep.load_agencyinfo()
# gtfs_status = data_prep.load_catalog_gtfs()

AttributeError: module 'data_prep' has no attribute 'load_vehiclesdata'

In [ ]:
vehicles.sample()

In [ ]:
agency_info

## Read in Catalog

In [ ]:
catalog = intake.open_catalog("catalog.yml")

In [ ]:
catalog

In [ ]:
gtfs_status = catalog.gtfs_status.read()

In [ ]:
gtfs_status = to_snakecase(gtfs_status)

In [ ]:
gtfs_status.head()

In [ ]:
gtfs_status.name.nunique()

## Read in Vehicle Data

In [ ]:
def load_vehiclesdata():
    vehicles_info =  pd.read_excel('gs://calitp-analytics-data/data-analyses/5311 /2020-Vehicles_1.xlsm',
                                   sheet_name = ['Age Distribution'])
    pd.set_option('display.max_columns', None)
    #cannot use to_snakecase because of integer column names
    vehicles_info = vehicles_info['Age Distribution']
    vehicles = (vehicles_info>>filter(_.State=='CA'))
    
    return vehicles

In [ ]:
vehicles = load_vehiclesdata()

In [ ]:
vehicles.sample(5)

* **note:** multiple entries for each agency

In [ ]:
len(vehicles)

In [ ]:
print(vehicles.Agency.nunique())
#print(vehicle_count.Agency.nunique())

## Read in Agency Information

In [ ]:
def load_agencyinfo(): 
    agencies = pd.concat(pd.read_excel('gs://calitp-analytics-data/data-analyses/5311 /2020_Agency_Information.xlsx',
                                      sheet_name=None),ignore_index=True)
    agencies = to_snakecase(agencies)
    agency_info = agencies>>filter(_.state=='CA')
    
    return agency_info

In [ ]:
agency_info = load_agencyinfo()

In [ ]:
agency_info.sample()

## Read in Agency Revenue

In [ ]:
agency_revenue = pd.concat(pd.read_excel('gs://calitp-analytics-data/data-analyses/5311 /2020_revenue_sources.xlsx',
                                      sheet_name=None), ignore_index=True)

In [ ]:
agency_revenue = to_snakecase(agency_revenue)

In [ ]:
agency_revenue.sample(5)

In [ ]:
agency_revenue.info()

* no state information. have to merge with other dataframes to get the information we need.

## Read BlackCat Data

In [ ]:
def load_grantprojects(): 
    df = pd.concat(pd.read_excel('gs://calitp-analytics-data/data-analyses/5311 /Grant_Projects.xlsx', sheet_name=None), ignore_index=True)
    
    pd.set_option('display.max_columns', None)
    
    #a bit of cleaning
    df = to_snakecase(df)
    df = df.drop(columns=['project_closed_by', 'project_closed_date', 'project_closed_time'])
    
    #get just the 5311 agencies
    df = (df>>filter(_.funding_program.str.contains('5311')))
    
    return df

In [ ]:
df = load_grantprojects()

In [ ]:
df.sample()

In [ ]:
len(df)

In [ ]:
df>>count(_.funding_program)

In [ ]:
df_agency = df>> count(_.organization_name) >> arrange(-_.n)

In [ ]:
print(len(df_agency.organization_name.unique()))

In [ ]:
df_agency.head()

In [ ]:
sorted(df_agency.organization_name.unique())

* no obvious duplicates or multiple names for one agency

In [ ]:
df >> group_by(_.organization_name) >> summarize(n=_.ali.nunique()) >> arrange(-_.n)

In [ ]:
df >> group_by(_.organization_name) >> summarize(n=_.grant_number.nunique()) >> arrange(-_.n)

## Looking at Agency Names and NTD IDs

In [ ]:
agency_crosswalk = agency_info[['agency_name','ntd_id']]

In [ ]:
agency_crosswalk>>count(_.agency_name)>>arrange(-_.n)>>filter(_.n>1)

In [ ]:
agency_crosswalk>>filter(_.agency_name=='Humboldt Transit Authority')

In [ ]:
#checking to see if the NTD ID is the same in the Vehicles dataset
(vehicles>>filter(_.Agency=='Humboldt Transit Authority'))

## Misc.